# It's a Machine and Natural Language Model

In [1]:
from src.IaMaN.base import LM
from src.utils.data import load_wikitext
from collections import defaultdict
from collections import Counter
from tqdm import tqdm
import numpy as np
import os, re
##
seed = 691; max_char = 200_000_000
m = 10; space = False; fine_tune = True; num_posttrain = 1000
positional = 'dependent'; positionally_encode = True; bits = 50
do_ife = False; update_ife = False; update_bow = True
runners = 25; gpu = True; tokenizer = 'hr-bpe'
##
##
total_posttrain = 0; ptdocs = []
if num_posttrain:
    num_posttrain = 0 if num_posttrain == 'all' else num_posttrain
    print("Loading WikiText-103 as post-training data...")
    posttrain_docs = load_wikitext(v = '103', split = "train", num_articles = num_posttrain, seed = seed, space = space, rebuild = True)
    ptdocs = [["".join(s) for s in d['document']] for d in posttrain_docs]; ptdocs = [d for d in ptdocs if len(d)]
print("Loading WikiText-2 LM'ing data...")
train_docs = load_wikitext(v = '2', split = "train", num_articles = 0, seed = seed, space = space, rebuild = True)
test_docs = load_wikitext(v = '2', split = "test", num_articles = 0, seed = seed, space = space, rebuild = True)
test_docs = [doc for doc in test_docs if len(doc['text']) and (len(doc['text']) <= max_char)]#[:2]
train_docs = [doc for doc in train_docs if len(doc['text']) and (len(doc['text']) <= max_char)]#[:10]
##
print('Avail. post-train, total post-train, Avail. gold, total gold-train, total test-gold: ', 
      total_posttrain, len(ptdocs), len(train_docs) + len(test_docs), len(train_docs), len(test_docs))

Loading WikiText-103 as post-training data...
Loading WikiText-2 LM'ing data...
Avail. post-train, total post-train, Avail. gold, total gold-train, total test-gold:  0 986 672 611 61


In [2]:
##
docs = [["".join(s) for s in d['document']] for d in train_docs]
tdocs = [["".join(s) for s in d['document']] for d in test_docs]
covering = [[re.split('( )', ''.join(s)) for s in d['document']] for d in train_docs]
tcovering = [[re.split('( )', ''.join(s)) for s in d['document']] for d in test_docs]
covering_vocab = set([t for d in covering for s in d for t in s])
##
model = LM(m = m, tokenizer = tokenizer, noise = 0.001, seed = seed, space = space, positional = positional,
           positionally_encode = positionally_encode, do_ife = do_ife, bits = bits,
           runners = runners, gpu = gpu)
data_streams = model.fit(docs, f'WT2-{len(test_docs)}', fine_tune = fine_tune, covering = covering)
model.post_train(ptdocs, update_ife = update_ife, update_bow = update_bow, fine_tune = fine_tune)
if fine_tune and ptdocs:
    model.fine_tune(docs, covering = covering, streams = data_streams)

Training tokenizer...


  0%|          | 0/611 [00:00<?, ?it/s]

Built a vocabulary of 24048 types
Tokenizing documents...



100%|██████████| 611/611 [07:31<00:00,  1.35it/s]


Counting documents and aggregating counts...


...


100%|██████████| 986/986 [00:15<00:00, 65.19it/s]


Building cipher... 


  0%|          | 4821/37586816 [00:00<12:59, 48199.82it/s]

 done.
Encoding parameters...



  3%|▎         | 989890/37586816 [00:27<19:13, 31732.45it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 46%|████▋     | 17405972/37586816 [08:24<09:28, 35468.93it/s]
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)


 81%|████████  | 30519769/37586816 [16:00<04:27, 26392.90it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the co

In [3]:
model.generate(m = 50, seed = seed)

ern with . 
 uneven withe to to name encouragee or . . , Polishe a 971spectively 66 foodon , in hadty ofport exte I and many . Although Gary end at post . @.@ an . Raye consider frome


In [4]:
out = model.stencil(docs = tdocs, return_output = True)


0it [00:00, ?it/s]
1it [00:00,  1.07it/s]
2it [00:02,  1.22s/it]
3it [00:03,  1.05s/it]
4it [00:06,  1.78s/it]
5it [00:07,  1.43s/it]
6it [00:08,  1.24s/it]
7it [00:14,  2.57s/it]
8it [00:14,  2.01s/it]
9it [00:20,  3.15s/it]
10it [00:21,  2.38s/it]
11it [00:22,  2.00s/it]
12it [00:25,  2.50s/it]
13it [00:26,  1.83s/it]
14it [00:26,  1.52s/it]
15it [00:27,  1.10s/it]
16it [00:27,  1.06it/s]
17it [00:30,  1.51s/it]
18it [00:30,  1.16s/it]
19it [00:32,  1.23s/it]
20it [00:33,  1.28s/it]
21it [00:35,  1.39s/it]
22it [00:35,  1.10s/it]
23it [00:36,  1.15s/it]
24it [00:42,  2.51s/it]
25it [00:43,  1.73s/it]

0it [00:00, ?it/s]
1it [00:02,  2.82s/it]
2it [00:07,  3.49s/it]
3it [00:08,  2.58s/it]
5it [00:09,  2.05s/it]
6it [00:10,  1.57s/it]
7it [00:12,  1.84s/it]
8it [00:13,  1.42s/it]
9it [00:17,  2.32s/it]
10it [00:20,  2.50s/it]
11it [00:21,  1.89s/it]
12it [00:29,  3.91s/it]
13it [00:31,  3.18s/it]
14it [00:35,  3.55s/it]
15it [00:40,  4.02s/it]
16it [00:49,  5.48s/it]
17it [00:49,  3.9

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 2.0 163.95 163.95 982 982



  3%|▎         | 2/61 [00:25<08:47,  8.94s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 3.0 234.24 234.24 5984 5984



  5%|▍         | 3/61 [00:34<08:42,  9.01s/it]

...

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  59 97.0 256.48 256.48 6381 6381



 98%|█████████▊| 60/61 [17:45<00:15, 15.44s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  60 98.0 199.32 199.32 4807 4807



100%|██████████| 61/61 [18:00<00:00, 17.72s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  61 100.0 274.79 274.79 4164 4164


In [5]:
import numpy as np
('micro: ', round(1/(10**(np.log10([p[0] for ps in out for p in ps]).mean())), 3),
 'macro: ', round(np.mean([1/(10**(np.log10([p[0] for p in ps]).mean())) for ps in out]), 3))

('micro: ', 238.949, 'macro: ', 242.292)